# Larsson Portfolio Analysis - Trading Reports Generator

## 📋 Notebook Overview
This notebook generates comprehensive trading reports based on dual-timeframe Larsson technical analysis.

## 🎯 Purpose
- Analyze tickers and baskets using custom Larsson state indicators
- Generate actionable buy/sell recommendations
- Export reports in multiple formats (PDF summary, PDF detailed, Excel)

## 📁 File Dependencies
- **stocks.txt** - List of tickers and basket definitions
- **holdings.csv** - Current positions (ticker, shares, avg_cost)
- **targets.csv** - Target allocations (ticker, target_pct, target_value)

## 📊 Output Files (saved to workspace/portfolio_results/)
1. **buy_summary_*.pdf** - Quick reference table of buy opportunities
2. **buy_detailed_*.pdf** - Deep analysis with multi-zone entries (1 page per ticker)
3. **sell_summary_*.pdf** - Quick reference table of capital protection actions
4. **sell_detailed_*.pdf** - Complete exit playbook (1 page per position)
5. **trading_actions_*.xlsx** - 3-sheet Excel workbook (Buy, Sell, Technical Data)

## 🚀 Quick Start
1. **Run Cell 1-2**: Load data and run analysis (~12 seconds)
2. **Run Cells 3-11**: Generate all reports and auto-cleanup
3. **Review Outputs**: Check portfolio_results/ folder for PDFs and Excel

## 🔄 Workflow
- **Morning Routine**: Run all cells to get fresh analysis
- **Quick Decisions**: Use Summary PDFs
- **Deep Research**: Use Detailed PDFs for high-conviction positions

## ⚙️ Configuration
- **starting_cash**: Adjust in Cells 4-10 (default: $104,967)
- **auto_archive**: Keeps 7 most recent report sets (Cell 11)

---

In [ ]:
# ===================================================================================================
# CELL 1: ENVIRONMENT SETUP & CONFIGURATION
# ===================================================================================================
# Purpose: Initialize workspace, import dependencies, and parse tickers from stocks.txt
# Inputs:  stocks.txt (list of tickers and basket definitions)
# Outputs: individual_tickers (list), baskets (dict), RESULTS_DIR (Path)
# ===================================================================================================

from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime
import pandas as pd
import importlib

# Import and reload script module (ensures latest code changes are used)
import technical_analysis
importlib.reload(technical_analysis)
from technical_analysis import parse_stocks_file, get_signal_description, calculate_reduction_amounts, get_reduction_tranches

# Set workspace root directory
ROOT = Path.cwd()
stocks_file = ROOT / 'stocks.txt'

# Create results directory for all outputs
RESULTS_DIR = ROOT / 'portfolio_results'
RESULTS_DIR.mkdir(exist_ok=True)
(RESULTS_DIR / 'archive').mkdir(exist_ok=True)

# Generate unified timestamp for all reports in this session
TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M")

# Parse stocks.txt to extract:
# - individual_tickers: standalone tickers (e.g., ['TSLA', 'NVDA', 'MSFT', ...])
# - baskets: grouped tickers for sector analysis (e.g., {'Main AI Basket': ['TSLA', 'NVDA', ...]})
individual_tickers, baskets = parse_stocks_file(stocks_file)

print(f"Individual tickers: {individual_tickers}")
print(f"\nBaskets: {baskets}")
print(f"\n✓ Results directory: {RESULTS_DIR}")
print(f"✓ Report timestamp: {TIMESTAMP}")

Individual tickers: ['TSLA', 'NVDA', 'MSFT', 'META', 'PLTR', 'MSTR', 'ASML', 'AMD', 'AVGO', 'ALAB', 'MRVL', 'BTC-USD', 'SOL-USD']

Baskets: {'Main AI Basket': ['TSLA', 'NVDA', 'MSFT', 'META', 'PLTR', 'ASML'], 'Secondary AI Basket': ['AMD', 'AVGO', 'ALAB', 'MRVL']}


In [3]:
# ===================================================================================================
# CELL 2: BATCH TECHNICAL ANALYSIS
# ===================================================================================================
# Purpose: Run parallel technical analysis on all tickers and baskets
# Inputs:  individual_tickers, baskets from Cell 1
# Outputs: df (DataFrame with signals, prices, technical levels, confluence ratings)
# Speed:   ~6 concurrent requests (reduces runtime from 60s to ~12s for 13 tickers)
# ===================================================================================================

from technical_analysis import analyze_ticker, analyze_basket

def run_batch(individual_tickers, baskets, concurrency=6, daily_bars=60, weekly_bars=52):
    """
    Run technical analysis on all tickers and baskets
    
    Args:
        individual_tickers: List of ticker symbols to analyze
        baskets: Dict of basket names -> constituent tickers
        concurrency: Number of parallel API requests (default: 6)
        daily_bars: Number of daily candles for analysis (default: 60)
        weekly_bars: Number of weekly candles for analysis (default: 52)
    
    Returns:
        DataFrame with columns: ticker, signal, current_price, confluence,
        recommendation, all technical levels (SMAs, support, resistance, volume profile)
    """
    results = []
    
    # Analyze individual tickers in parallel (significant speed boost)
    with ThreadPoolExecutor(max_workers=concurrency) as ex:
        futures = {ex.submit(analyze_ticker, t, daily_bars, weekly_bars): t for t in individual_tickers}
        for fut in as_completed(futures):
            results.append(fut.result())
    
    # Analyze baskets sequentially (market cap weighted aggregations of constituents)
    for basket_name, constituents in baskets.items():
        basket_result = analyze_basket(basket_name, constituents, daily_bars, weekly_bars)
        results.append(basket_result)
    
    return pd.DataFrame(results)

# Execute batch analysis
df = run_batch(individual_tickers, baskets)
df = df.round(2)  # Round all numeric values to 2 decimal places for readability

# Sort by ticker alphabetically (baskets at end with brackets)
df = df.sort_values('ticker', key=lambda x: x.str.replace('[', '~').str.replace(']', ''))
df = df.reset_index(drop=True)

# Print summary statistics
print(f"\nAnalysis complete: {len(df)} rows ({len(individual_tickers)} tickers + {len(baskets)} baskets)")
print("FULL HOLD + ADD tickers:", ', '.join(df[df['signal'] == 'FULL HOLD + ADD']['ticker']))

df


Analysis complete: 15 rows (13 tickers + 2 baskets)
FULL HOLD + ADD tickers: ASML, MRVL, NVDA, PLTR, TSLA, [Main AI Basket]


,ticker,signal,current_price,price_note,date,d20,d50,d100,d200,w10,...,weekly_val,s1,s2,s3,r1,r2,r3,notes,confluence,recommendation
0,ALAB,HOLD MOST → REDUCE,170.11,last close,2026-01-05,163.28,161.06,179.52,135.07,160.05,...,103.23,148.51,131.42,84.78,262.90,201.86,199.47,,EXTENDED,No Buy
1,AMD,HOLD,223.18,last close,2026-01-05,214.41,227.63,203.23,163.23,220.54,...,76.48,194.28,161.81,153.34,267.08,227.30,NaN,,BALANCED,No Buy
2,ASML,FULL HOLD + ADD,1224.42,last close,2026-01-05,1088.93,1060.43,970.31,845.81,1074.78,...,574.25,988.40,946.11,933.75,NaN,NaN,NaN,,WEAK,Skip – Poor Setup
3,AVGO,HOLD,340.26,last close,2026-01-05,358.64,361.23,345.20,289.66,356.01,...,214.71,328.43,323.43,280.85,413.82,385.74,372.88,,BALANCED,No Buy
4,BTC-USD,HOLD MOST → REDUCE,93688.40,last close,2026-01-05,88391.22,89207.40,99764.85,106648.38,91623.63,...,92035.55,91242.89,89260.10,84436.31,126198.07,124457.12,123091.61,,EXTENDED,No Buy
5,META,HOLD MOST → REDUCE,662.19,last close,2026-01-05,658.65,650.41,697.18,673.01,642.21,...,579.95,580.78,578.18,546.88,795.06,789.62,758.54,,EXTENDED,No Buy
6,MRVL,FULL HOLD + ADD,91.64,last close,2026-01-05,87.43,87.81,82.50,74.46,87.90,...,59.80,79.06,73.62,69.71,127.15,125.76,102.77,,EXTENDED,Wait for Support
7,MSFT,HOLD MOST → REDUCE,474.75,last close,2026-01-05,482.87,494.31,502.51,477.17,485.14,...,406.12,464.89,407.71,404.37,553.50,552.69,530.04,,BALANCED,No Buy
8,MSTR,CASH,163.06,last close,2026-01-05,167.75,202.40,267.75,320.20,178.86,...,254.29,155.61,120.23,113.69,543.00,457.22,430.35,,BALANCED,No Buy
9,NVDA,FULL HOLD + ADD,189.42,last close,2026-01-05,183.26,186.69,183.08,160.84,184.14,...,86.60,176.75,169.54,164.05,212.18,NaN,NaN,,EXTENDED,Wait for Support


In [4]:
# ===================================================================================================
# CELL 3: PRINT BUY SUMMARY (Console Output)
# ===================================================================================================
# Purpose: Display markdown-formatted buy opportunities in console
# Inputs:  df (from Cell 2), targets.csv (optional), cash_available (adjustable)
# Outputs: Markdown table printed to console
# Note:    This is console output only - PDF export is in Cell 4
# ===================================================================================================

def print_buy_summary(df, cash_available=118305):
    print("\n### Larsson Portfolio Buy Summary – Next Trading Day\n")
    print("**Rule-Based Only** — Conservative phased entry rules:")
    print("- Primary adds only on pullbacks to predefined support zones.\n")
    
    print(f"**Cash Available**: ~${cash_available:,.0f} (~59% dry powder)\n")
    
    # Load target allocations from targets.csv
    targets_file = ROOT / 'targets.csv'
    if targets_file.exists():
        targets_df = pd.read_csv(targets_file)
        target_dict = dict(zip(targets_df['ticker'], targets_df['target_pct']))
        value_dict = dict(zip(targets_df['ticker'], targets_df['target_value']))
    else:
        target_dict = {}
        value_dict = {}
        print("⚠️  targets.csv not found - using N/A for target percentages\n")
    
    # Use exact match instead of contains
    eligible = df[df['signal'] == "FULL HOLD + ADD"]
    if eligible.empty:
        print("No FULL HOLD + ADD names — no buys recommended.")
        return
    
    def get_primary_zone(row):
        """Hybrid conservative primary add zone: Lower Value Area + Key Long-Term SMAs"""
        val = row['daily_val']
        poc = row['daily_poc']
        d100 = row['d100']
        d200 = row['d200']
        
        # Handle NaN/missing
        if pd.isna(val) or pd.isna(poc):
            return f"Near Key SMAs (D100 ${int(d100)} / D200 ${int(d200)})"
        
        lower_va = f"Lower Value Area (${int(val)}–${int(poc)})"
        sma_part = f"or Key Long-Term SMA (D100 ${int(d100)} / D200 ${int(d200)})"
        return f"{lower_va} {sma_part}"
    
    print("| Ticker | Target % | Current Price (Close) | Confluence | Buy Recommendation | Primary Add (40–50% of target) | Primary Add Zone (Conservative) | Approx Shares at Zone |")
    print("|--------|----------|-----------------------|------------|------------------------------|--------------------------------|---------------------------------|-----------------------|")
    
    for _, row in eligible.iterrows():
        ticker = row['ticker']
        price = row['current_price']
        confluence = row['confluence']
        rec = row['recommendation']
        
        # Get target % and value from config file
        target_pct = target_dict.get(ticker, 'N/A')
        target_val = value_dict.get(ticker, 4001)
        
        if target_pct != 'N/A':
            target_pct_str = f"{target_pct}%"
        else:
            target_pct_str = 'N/A'
        
        # Calculate primary add amounts (40-50% of target)
        primary_low = target_val * 0.4
        primary_high = target_val * 0.5
        
        # Get primary zone
        zone = get_primary_zone(row)
        
        # Share estimate using approximate zone midpoint (or current price fallback)
        zone_mid_est = price * 0.9  # rough 10% dip estimate for conservatism
        shares_low = int(primary_low / zone_mid_est)
        shares_high = int(primary_high / zone_mid_est)
        shares_str = f"{shares_low}–{shares_high} shares"
        
        print(f"| **{ticker}** | {target_pct_str} | ${price:.2f} | **{confluence}** | {rec} | ~${primary_low:,.0f}–${primary_high:,.0f} | {zone} | {shares_str} |")
    
    print("\n**No Buy Action**")
    print("- All other names: Not FULL HOLD + ADD or confluence insufficient.\n")
    print("**Execution Plan**")
    print("- No limits to place / Wait for weakness to primary zones / etc.")

# Execute buy summary print
print_buy_summary(df)


### Larsson Portfolio Buy Summary – Next Trading Day

**Rule-Based Only** — Conservative phased entry rules:
- Primary adds only on pullbacks to predefined support zones.

**Cash Available**: ~$118,305 (~59% dry powder)

| Ticker | Target % | Current Price (Close) | Confluence | Buy Recommendation | Primary Add (40–50% of target) | Primary Add Zone (Conservative) | Approx Shares at Zone |
|--------|----------|-----------------------|------------|------------------------------|--------------------------------|---------------------------------|-----------------------|
| **ASML** | 5% | $1224.42 | **WEAK** | Skip – Poor Setup | ~$4,000–$5,000 | Lower Value Area ($964–$1028) or Key Long-Term SMA (D100 $970 / D200 $845) | 3–4 shares |
| **MRVL** | 2% | $91.64 | **EXTENDED** | Wait for Support | ~$1,600–$2,000 | Lower Value Area ($81–$89) or Key Long-Term SMA (D100 $82 / D200 $74) | 19–24 shares |
| **NVDA** | 18% | $189.42 | **EXTENDED** | Wait for Support | ~$14,402–$18,002 | Lower Value 

In [5]:
# ===================================================================================================
# CELL 6: PRINT SELL SUMMARY (Console Output)
# ===================================================================================================
# Purpose: Display markdown-formatted capital protection actions in console
# Inputs:  df (from Cell 2), holdings.csv, starting_cash (adjustable)
# Outputs: Markdown table printed to console
# Note:    Only shows positions with defensive signals AND current holdings
# ===================================================================================================

def print_sell_summary(df, starting_cash=118305):
    print("\n### ⚠️ Capital Protection Summary – Positions Requiring Reduction\n")
    print("**Defensive Actions Required** — Asymmetric defense protocol:")
    print("- Quick to defend on bearish turns, patient to re-enter on bullish recovery")
    print("- Phased exits into strength/rallies (never panic sell at lows)")
    print("- Proceeds to cash — redeploy only when signals improve to FULL HOLD + ADD\n")
    
    # Load current holdings
    holdings_file = ROOT / 'holdings.csv'
    holdings_dict = {}
    if holdings_file.exists():
        holdings_df = pd.read_csv(holdings_file)
        for _, row in holdings_df.iterrows():
            holdings_dict[row['ticker']] = {
                'shares': row['shares'],
                'avg_cost': row['avg_cost']
            }
    else:
        print("⚠️  holdings.csv not found - cannot generate sell summary\n")
        return
    
    # Calculate total portfolio value
    total_holdings_value = 0
    for ticker, holding in holdings_dict.items():
        if holding['shares'] > 0:
            current_price = df[df['ticker'] == ticker]['current_price'].values
            if len(current_price) > 0:
                total_holdings_value += holding['shares'] * current_price[0]
    
    total_portfolio_value = total_holdings_value + starting_cash
    
    print(f"**Total Portfolio Value**: ${total_portfolio_value:,.0f}")
    print(f"**Current Holdings**: ${total_holdings_value:,.0f}")
    print(f"**Cash Available**: ${starting_cash:,.0f}\n")
    
    # Filter for positions requiring capital protection
    defensive_signals = ["HOLD MOST → REDUCE", "REDUCE", "LIGHT / CASH", "CASH", "FULL CASH / DEFEND"]
    positions_to_reduce = df[
        (df['signal'].isin(defensive_signals)) & 
        (~df['ticker'].str.startswith('['))  # Exclude baskets
    ]
    
    if positions_to_reduce.empty:
        print("✅ No defensive signals detected — capital protection not required.")
        print("All positions remain in FULL HOLD + ADD, HOLD, or SCALE IN mode.\n")
        return
    
    # Count positions held that need reduction
    positions_with_holdings = []
    for _, row in positions_to_reduce.iterrows():
        ticker = row['ticker']
        holding = holdings_dict.get(ticker, {'shares': 0, 'avg_cost': 0})
        if holding['shares'] > 0:
            positions_with_holdings.append(row)
    
    if not positions_with_holdings:
        print("✅ Defensive signals detected but no current holdings require action.\n")
        return
    
    print("| Ticker | Signal | Current Value | Reduce % | Tranche 1 (Immediate) | Zone 1 | Tranche 2 (On Bounce) | Zone 2 | Keep |")
    print("|--------|--------|---------------|----------|-----------------------|--------|-----------------------|--------|------|")
    
    for row in positions_with_holdings:
        ticker = row['ticker']
        signal = row['signal']
        price = row['current_price']
        
        # Get holding
        holding = holdings_dict.get(ticker, {'shares': 0, 'avg_cost': 0})
        current_value = holding['shares'] * price
        
        # Calculate reduction amounts
        reduction_amount, keep_amount, reduction_pct = calculate_reduction_amounts(signal, current_value)
        
        # Get tranches
        tranches = get_reduction_tranches(signal, reduction_pct, reduction_amount)
        
        # Get resistance levels for sell zones
        r1 = row['r1'] if not pd.isna(row['r1']) else price * 1.02
        r2 = row['r2'] if not pd.isna(row['r2']) else price * 1.05
        r3 = row['r3'] if not pd.isna(row['r3']) else price * 1.08
        
        # Format tranches for display
        if len(tranches) == 1:
            # Single tranche
            tranche1_amount, tranche1_pct, timing1 = tranches[0]
            tranche1_shares = int(holding['shares'] * tranche1_pct)
            
            zone1 = f"${int(r1)}+ or current"
            
            print(f"| **{ticker}** | {signal} | ${current_value:,.0f} | {int(reduction_pct*100)}% | ${tranche1_amount:,.0f} ({tranche1_shares} sh) | {zone1} | — | — | ${keep_amount:,.0f} |")
        
        elif len(tranches) == 2:
            # Two tranches
            tranche1_amount, tranche1_pct, timing1 = tranches[0]
            tranche2_amount, tranche2_pct, timing2 = tranches[1]
            tranche1_shares = int(holding['shares'] * tranche1_pct)
            tranche2_shares = int(holding['shares'] * tranche2_pct)
            
            zone1 = f"${int(r1)} or current"
            zone2 = f"${int(r2)}+ (on rally)"
            
            print(f"| **{ticker}** | {signal} | ${current_value:,.0f} | {int(reduction_pct*100)}% | ${tranche1_amount:,.0f} ({tranche1_shares} sh) | {zone1} | ${tranche2_amount:,.0f} ({tranche2_shares} sh) | {zone2} | ${keep_amount:,.0f} |")
        
        else:  # Three tranches (FULL CASH / DEFEND)
            tranche1_amount, tranche1_pct, timing1 = tranches[0]
            tranche2_amount, tranche2_pct, timing2 = tranches[1]
            tranche3_amount, tranche3_pct, timing3 = tranches[2]
            tranche1_shares = int(holding['shares'] * tranche1_pct)
            tranche2_shares = int(holding['shares'] * tranche2_pct)
            
            zone1 = f"${int(r1)} or current"
            zone2 = f"${int(r2)}+ / R3 ${int(r3)}"
            
            # Combine tranche 2 and 3 for display
            combined_t2_amount = tranche2_amount + tranche3_amount
            combined_t2_shares = tranche2_shares + int(holding['shares'] * tranche3_pct)
            
            print(f"| **{ticker}** | {signal} | ${current_value:,.0f} | {int(reduction_pct*100)}% | ${tranche1_amount:,.0f} ({tranche1_shares} sh) | {zone1} | ${combined_t2_amount:,.0f} ({combined_t2_shares} sh) | {zone2} | ${keep_amount:,.0f} |")
    
    print("\n**Execution Guidelines**")
    print("- Tranche 1: Place limit orders at Zone 1 prices or sell at market")
    print("- Tranche 2: Wait for bounce to Zone 2 resistance levels")
    print("- Never panic sell at lows — use rallies to exit at better prices")
    print("- Re-entry only after weekly Larsson state reclaims bullish (+1)\n")

# Execute sell summary print
print_sell_summary(df, starting_cash=104967)


### ⚠️ Capital Protection Summary – Positions Requiring Reduction

**Defensive Actions Required** — Asymmetric defense protocol:
- Quick to defend on bearish turns, patient to re-enter on bullish recovery
- Phased exits into strength/rallies (never panic sell at lows)
- Proceeds to cash — redeploy only when signals improve to FULL HOLD + ADD

**Total Portfolio Value**: $189,898
**Current Holdings**: $84,931
**Cash Available**: $104,967

| Ticker | Signal | Current Value | Reduce % | Tranche 1 (Immediate) | Zone 1 | Tranche 2 (On Bounce) | Zone 2 | Keep |
|--------|--------|---------------|----------|-----------------------|--------|-----------------------|--------|------|
| **BTC-USD** | HOLD MOST → REDUCE | $46,844 | 20% | $9,369 (0 sh) | $126198+ or current | — | — | $37,475 |

**Execution Guidelines**
- Tranche 1: Place limit orders at Zone 1 prices or sell at market
- Tranche 2: Wait for bounce to Zone 2 resistance levels
- Never panic sell at lows — use rallies to exit at better 

In [ ]:
# ===================================================================================================
# CELL 4: EXPORT BUY SUMMARY PDF (Quick Reference)
# ===================================================================================================
# Purpose: Generate single-page summary table of all buy opportunities
# Inputs:  df (from Cell 2), holdings.csv, targets.csv
# Outputs: buy_summary_YYYYMMDD_HHMM.pdf in ~/Downloads
# Format:  Compact table with target %, current %, confluence, zones, share estimates
# Use:     Quick morning review - which tickers to buy and at what prices
# ===================================================================================================

from reportlab.lib import colors
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
from reportlab.lib.enums import TA_CENTER
from datetime import datetime, timedelta

def export_buy_summary_to_pdf(df, starting_cash=118305):
    """Generate a styled PDF report of the buy summary using reportlab"""
    
    # Load target allocations
    targets_file = ROOT / 'targets.csv'
    if targets_file.exists():
        targets_df = pd.read_csv(targets_file)
        target_dict = dict(zip(targets_df['ticker'], targets_df['target_pct']))
        value_dict = dict(zip(targets_df['ticker'], targets_df['target_value']))
    else:
        target_dict = {}
        value_dict = {}
    
    # Load current holdings
    holdings_file = ROOT / 'holdings.csv'
    holdings_dict = {}
    if holdings_file.exists():
        holdings_df = pd.read_csv(holdings_file)
        for _, row in holdings_df.iterrows():
            holdings_dict[row['ticker']] = {
                'shares': row['shares'],
                'avg_cost': row['avg_cost']
            }
    
    # Holdings are tracked for allocation %, but cash available is separate
    # (holdings already paid for separately)
    cash_available = starting_cash
    
    # Get eligible tickers (exclude baskets)
    eligible = df[(df['signal'] == "FULL HOLD + ADD") & (~df['ticker'].str.startswith('['))]
    
    # Calculate next trading day (skip weekends)
    today = datetime.now()
    next_trading_day = today + timedelta(days=1)
    # Skip Saturday (5) and Sunday (6)
    while next_trading_day.weekday() >= 5:
        next_trading_day += timedelta(days=1)
    
    # Generate PDF filename - save directly to Downloads (no timestamp, will overwrite)
    # Generate PDF filename with unified timestamp - save to workspace
    pdf_filename = f"buy_summary_{TIMESTAMP}.pdf"
    pdf_path = RESULTS_DIR / pdf_filename
    
    # Create PDF
    doc = SimpleDocTemplate(str(pdf_path), pagesize=letter,
                           rightMargin=30, leftMargin=30,
                           topMargin=30, bottomMargin=18)
    
    # Container for the 'Flowable' objects
    elements = []
    
    # Define styles
    styles = getSampleStyleSheet()
    title_style = ParagraphStyle(
        'CustomTitle',
        parent=styles['Heading1'],
        fontSize=18,
        textColor=colors.HexColor('#2c3e50'),
        spaceAfter=20,
        alignment=TA_CENTER
    )
    
    # Add title with dates
    title_text = f"Larsson Portfolio Buy Summary – For {next_trading_day.strftime('%A, %B %d, %Y')}<br/><font size=12>(Data as of {today.strftime('%B %d')} Close)</font>"
    elements.append(Paragraph(title_text, title_style))
    elements.append(Spacer(1, 0.2*inch))
    
    # Calculate total portfolio value
    total_holdings_value = 0
    for ticker, holding in holdings_dict.items():
        if holding['shares'] > 0:
            current_price = df[df['ticker'] == ticker]['current_price'].values
            if len(current_price) > 0:
                total_holdings_value += holding['shares'] * current_price[0]
    
    total_portfolio_value = total_holdings_value + cash_available
    cash_pct = (cash_available / total_portfolio_value * 100) if total_portfolio_value > 0 else 0
    
    # Add summary info
    summary_text = f"""
    <b>Rule-Based Only</b> — Conservative phased entry rules:<br/>
    <br/>
    • <b>EXTENDED</b> Stocks: Wait for pullback to support (D100/D200)<br/>
    • <b>BALANCED</b> Stocks: Enter on dip to lower Value Area or key SMAs<br/>
    • <b>WEAK</b> Stocks: Skip until technical setup improves<br/>
    <br/>
    <b>Total Portfolio Value:</b> ${total_portfolio_value:,.0f}<br/>
    <b>Current Holdings:</b> ${total_holdings_value:,.0f}<br/>
    <b>Cash Available:</b> ${cash_available:,.0f} (~{cash_pct:.0f}% dry powder)
    """
    elements.append(Paragraph(summary_text, styles['Normal']))
    elements.append(Spacer(1, 0.3*inch))
    
    # Add Portfolio Health Check section (baskets as macro indicators)
    basket_rows = df[df['ticker'].str.startswith('[')]
    if not basket_rows.empty:
        basket_text = "<b>Portfolio Health Check (Macro View)</b><br/>"
        for _, basket_row in basket_rows.iterrows():
            basket_name = basket_row['ticker'].strip('[]')
            signal = basket_row.get('signal', 'UNKNOWN')
            
            # Get constituents from baskets dict
            constituents = baskets.get(basket_name, [])
            constituents_str = ', '.join(constituents) if constituents else 'N/A'
            
            # Get signal description
            description = get_signal_description(signal)
            
            basket_text += f"• <b>{basket_name}</b> ({constituents_str}): <b>{signal}</b><br/>"
            basket_text += f"<font size=9><i>{description}</i></font><br/><br/>"
        
        elements.append(Paragraph(basket_text, styles['Normal']))
        elements.append(Spacer(1, 0.3*inch))
    
    # Prepare table data with holdings awareness
    table_data = [['Ticker', 'Target %', 'Current %', 'Price', 'Confluence', 'Recommendation', 'Next Add', 'Zone', 'Shares']]
    if eligible.empty:
        # Add a "No Buy" row if no eligible tickers
        table_data.append(['No Eligible', 'N/A', 'N/A', 'N/A', 'N/A', 'No FULL HOLD + ADD signals', 'N/A', 'N/A', 'N/A'])
    else:
        for _, row in eligible.iterrows():
            ticker = row['ticker']
            price = row['current_price']
            confluence = row['confluence']
            rec = row['recommendation']
            
            # Get target data - calculate based on total portfolio value
            target_pct = target_dict.get(ticker, 'N/A')
            if target_pct != 'N/A':
                target_val = (target_pct / 100) * total_portfolio_value
            else:
                target_val = 4001
            target_pct_str = f"{target_pct}%" if target_pct != 'N/A' else 'N/A'
            
            # Get current holding
            holding = holdings_dict.get(ticker, {'shares': 0, 'avg_cost': 0})
            current_value = holding['shares'] * price
            current_pct = (current_value / total_portfolio_value) * 100 if total_portfolio_value > 0 else 0
            current_pct_str = f"{current_pct:.0f}%" if current_pct > 0 else "0%"
            
            # Calculate remaining gap to target
            remaining_gap = target_val - current_value
            
            # Calculate primary add amounts (40-50% of remaining gap, not total target)
            if remaining_gap > 0:
                primary_low = remaining_gap * 0.4
                primary_high = remaining_gap * 0.5
            else:
                # Already at or above target
                primary_low = 0
                primary_high = 0
            
            # Shorten recommendation if too long (updated for new confluence/recommendation labels)
            rec_short = rec.replace("Wait for Support", "Wait Support").replace("Enter on Dip", "Enter on Dip").replace("Skip – Poor Setup", "Skip")
            
            # For "Skip" recommendations, don't show entry zones or share counts
            if rec_short == "Skip":
                primary_add_str = "N/A"
                zone = "Poor Setup"
                shares_str = "N/A"
            else:
                # Get primary zone based on confluence
                val = row['daily_val']
                poc = row['daily_poc']
                d100 = row['d100']
                d200 = row['d200']
                
                # Different zones based on confluence state
                if confluence == "BALANCED":
                    # For balanced, use basket signal to determine zone preference
                    # Find which basket(s) contain this ticker
                    ticker_baskets = [name for name, constituents in baskets.items() if ticker in constituents]
                    
                    # Check basket strength
                    strong_basket = False
                    if ticker_baskets:
                        for basket_name in ticker_baskets:
                            basket_signal = df[df['ticker'] == f'[{basket_name}]']['signal'].values
                            if len(basket_signal) > 0 and basket_signal[0] == "FULL HOLD + ADD":
                                strong_basket = True
                                break
                    
                    # For balanced with strong basket, D100 is acceptable
                    # For balanced with weak/no basket, prefer Lower VA for more safety
                    if strong_basket:
                        zone = f">> D100 ${int(d100)}"
                        zone_mid_est = d100
                    else:
                        if not pd.isna(val) and val < price:
                            zone = f">> Lower VA ${int(val)}\nor D100 ${int(d100)}"
                            zone_mid_est = val  # Target lower VA
                        else:
                            zone = f"D100 ${int(d100)}"
                            zone_mid_est = d100
                elif confluence == "EXTENDED":
                    # For extended, ALWAYS prefer D200 - extended means technically stretched
                    # Basket strength = conviction to wait patiently, not justification to chase
                    zone = f">> D200 ${int(d200)}\nor D100 ${int(d100)}"
                    zone_mid_est = d200  # Always target deeper pullback for extended stocks
                else:
                    # Fallback for other states
                    if pd.isna(val) or pd.isna(poc):
                        zone = f"D100 ${int(d100)}"
                        zone_mid_est = d100
                    else:
                        zone = f"${int(val)}-${int(poc)}"
                        zone_mid_est = (val + poc) / 2
                
                # Share estimate
                shares_low = int(primary_low / zone_mid_est)
                shares_high = int(primary_high / zone_mid_est)
                shares_str = f"{shares_low}-{shares_high}"
                primary_add_str = f"${primary_low:,.0f}-\n${primary_high:,.0f}"
            
            table_data.append([
                ticker,
                target_pct_str,
                current_pct_str,
                f"${price:.2f}",
                confluence,
                rec_short,
                primary_add_str,
                zone,
                shares_str
            ])
    
    # Create table with updated column widths for new Current % column
    table = Table(table_data, colWidths=[0.9*inch, 0.65*inch, 0.65*inch, 0.55*inch, 0.8*inch, 1.3*inch, 0.75*inch, 1.0*inch, 0.5*inch])
    
    # Apply table style with confluence color coding
    style_commands = [
        ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#3498db')),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 9),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.white),
        ('GRID', (0, 0), (-1, -1), 1, colors.grey),
        ('FONTSIZE', (0, 1), (-1, -1), 7),
        ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
    ]
    
    # Add confluence color coding for each row
    if not eligible.empty:
        for i, (_, row) in enumerate(eligible.iterrows(), start=1):
            confluence = row['confluence']
            if confluence == 'EXTENDED':
                color = colors.lightgreen
            elif confluence == 'BALANCED':
                color = colors.lightyellow
            elif confluence == 'WEAK':
                color = colors.lightcoral
            else:
                color = colors.white
            
            # Apply color to confluence column (column 4, shifted due to new Current % column)
            style_commands.append(('BACKGROUND', (4, i), (4, i), color))
    
    table.setStyle(TableStyle(style_commands))
    
    elements.append(table)
    elements.append(Spacer(1, 0.3*inch))
    
    # Add execution plan
    exec_text = """
    <b>Execution Plan</b><br/>
    <br/>
    • Place limit orders only at specified Zone prices (no chasing current prices)<br/>
    • Wait for pullbacks to target zones before entering positions<br/>
    • Other stocks: No action (insufficient signal strength or poor technical setup)
    """
    elements.append(Paragraph(exec_text, styles['Normal']))
    elements.append(Spacer(1, 0.2*inch))
    
    # Add footer
    footer_text = f"<font size=8>Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}</font>"
    elements.append(Paragraph(footer_text, styles['Normal']))
    
    # Build PDF
    doc.build(elements)
    print(f"✅ Buy Summary PDF saved to Downloads: {pdf_path}")

# Execute PDF generation
export_buy_summary_to_pdf(df, starting_cash=104967)

✅ Buy Summary PDF saved to Downloads: C:\Users\karms\Downloads\buy_summary_20260105_1758.pdf


In [ ]:
# ===================================================================================================
# CELL 5: EXPORT BUY DETAILED PDF (Deep Analysis)
# ===================================================================================================
# Purpose: Generate multi-page detailed analysis - ONE PAGE PER TICKER
# Inputs:  df (from Cell 2), holdings.csv, targets.csv
# Outputs: buy_detailed_YYYYMMDD_HHMM.pdf in ~/Downloads
# Format:  Cover page + 1 page per ticker with:
#          - 3-zone entry strategy (Primary/Secondary/Final tranches)
#          - Complete technical landscape (all SMAs, VP, S/R levels)
#          - Risk management (stops, position sizing, max loss)
#          - Basket context & conviction levels
#          - Confluence explanation (why EXTENDED/BALANCED/WEAK)
# Use:     Deep dive before making large position entries
# ===================================================================================================

from reportlab.platypus import PageBreak, KeepTogether

def export_buy_detailed_to_pdf(df, starting_cash=104967):
    """Generate detailed multi-page PDF with one page per buy opportunity"""
    
    # Load holdings and targets
    holdings_file = ROOT / 'holdings.csv'
    targets_file = ROOT / 'targets.csv'
    
    holdings_dict = {}
    if holdings_file.exists():
        holdings_df = pd.read_csv(holdings_file)
        for _, row in holdings_df.iterrows():
            holdings_dict[row['ticker']] = {'shares': row['shares'], 'avg_cost': row['avg_cost']}
    
    target_dict = {}
    value_dict = {}
    if targets_file.exists():
        targets_df = pd.read_csv(targets_file)
        target_dict = dict(zip(targets_df['ticker'], targets_df['target_pct']))
        value_dict = dict(zip(targets_df['ticker'], targets_df['target_value']))
    
    # Calculate portfolio value
    total_holdings_value = 0
    for ticker, holding in holdings_dict.items():
        if holding['shares'] > 0:
            current_price = df[df['ticker'] == ticker]['current_price'].values
            if len(current_price) > 0:
                total_holdings_value += holding['shares'] * current_price[0]
    
    total_portfolio_value = total_holdings_value + starting_cash
    
    # Get eligible tickers and sort alphabetically
    eligible = df[(df['signal'] == "FULL HOLD + ADD") & (~df['ticker'].str.startswith('['))]
    eligible = eligible.sort_values('ticker').reset_index(drop=True)
    
    if eligible.empty:
        print("✅ No buy opportunities - skipping detailed PDF")
        return
    
    # Generate PDF
    today = datetime.now()
    next_trading_day = today + timedelta(days=1)
    while next_trading_day.weekday() >= 5:
        next_trading_day += timedelta(days=1)
    
    pdf_filename = f"buy_detailed_{TIMESTAMP}.pdf"
    pdf_path = RESULTS_DIR / pdf_filename
    
    doc = SimpleDocTemplate(str(pdf_path), pagesize=letter,
                           rightMargin=30, leftMargin=30,
                           topMargin=30, bottomMargin=30)
    
    elements = []
    styles = getSampleStyleSheet()
    
    # Custom styles
    ticker_title_style = ParagraphStyle(
        'TickerTitle',
        parent=styles['Heading1'],
        fontSize=24,
        textColor=colors.HexColor('#2c3e50'),
        spaceAfter=10,
        alignment=TA_CENTER
    )
    
    section_style = ParagraphStyle(
        'Section',
        parent=styles['Heading2'],
        fontSize=14,
        textColor=colors.HexColor('#3498db'),
        spaceAfter=8,
        spaceBefore=12
    )
    
    # Cover page
    cover_title = f"Buy Opportunities - Detailed Analysis<br/><font size=14>For {next_trading_day.strftime('%A, %B %d, %Y')}</font>"
    elements.append(Paragraph(cover_title, ticker_title_style))
    elements.append(Spacer(1, 0.3*inch))
    
    summary = f"""
    <b>Portfolio Context:</b><br/>
    Total Portfolio Value: ${total_portfolio_value:,.0f}<br/>
    Cash Available: ${starting_cash:,.0f}<br/>
    <br/>
    <b>Opportunities Found: {len(eligible)}</b><br/>
    {', '.join(eligible['ticker'].tolist())}<br/>
    <br/>
    <i>Each ticker analyzed on separate page with multi-zone entry strategy,
    complete technical levels, risk parameters, and basket context.</i>
    """
    elements.append(Paragraph(summary, styles['Normal']))
    elements.append(PageBreak())
    
    # One page per ticker
    for idx, (_, row) in enumerate(eligible.iterrows()):
        ticker = row['ticker']
        price = row['current_price']
        confluence = row['confluence']
        rec = row['recommendation']
        
        # Get technical levels
        d20, d50, d100, d200 = row['d20'], row['d50'], row['d100'], row['d200']
        w10, w20, w200 = row['w10'], row['w20'], row['w200']
        daily_val, daily_poc, daily_vah = row['daily_val'], row['daily_poc'], row['daily_vah']
        weekly_val, weekly_poc, weekly_vah = row['weekly_val'], row['weekly_poc'], row['weekly_vah']
        s1, s2, s3 = row['s1'], row['s2'], row['s3']
        r1, r2, r3 = row['r1'], row['r2'], row['r3']
        
        # Target data
        target_pct = target_dict.get(ticker, 0)
        target_val = (target_pct / 100) * total_portfolio_value if target_pct > 0 else 0
        holding = holdings_dict.get(ticker, {'shares': 0, 'avg_cost': 0})
        current_val = holding['shares'] * price
        gap = target_val - current_val
        
        # === TICKER HEADER ===
        elements.append(Paragraph(f"{ticker}", ticker_title_style))
        elements.append(Paragraph(f"<b>Signal:</b> {row['signal']} | <b>Confluence:</b> {confluence}", styles['Normal']))
        elements.append(Spacer(1, 0.15*inch))
        
        # === SECTION: Position Sizing ===
        section_elements = []
        section_elements.append(Paragraph("Position Sizing & Entry Tranches", section_style))
        
        position_text = f"""
        <b>Target Allocation:</b> {target_pct}% (${target_val:,.0f})<br/>
        <b>Current Allocation:</b> {(current_val/total_portfolio_value*100):.1f}% (${current_val:,.0f})<br/>
        <b>Remaining Gap:</b> ${max(0, gap):,.0f}<br/>
        """
        section_elements.append(Paragraph(position_text, styles['Normal']))
        section_elements.append(Spacer(1, 0.1*inch))
        
        # Multi-zone entry strategy
        remaining_gap = max(0, gap)
        if remaining_gap > 0:
            # Zone 1: Primary (40-50% of gap)
            zone1_low = remaining_gap * 0.4
            zone1_high = remaining_gap * 0.5
            
            # Zone 2: Secondary (25-30% of gap)
            zone2_low = remaining_gap * 0.25
            zone2_high = remaining_gap * 0.3
            
            # Zone 3: Final (20-25% of gap)
            zone3_low = remaining_gap * 0.2
            zone3_high = remaining_gap * 0.25
            
            # Determine zone prices based on confluence
            if confluence == 'EXTENDED':
                zone1_price = d200
                zone1_label = f"D200 ${int(d200)}"
                zone2_price = s1 if not pd.isna(s1) else d200 * 0.95
                zone2_label = f"S1 ${int(zone2_price)}" if not pd.isna(s1) else f"Below D200 ${int(zone2_price)}"
                zone3_price = s2 if not pd.isna(s2) else d200 * 0.90
                zone3_label = f"S2 ${int(zone3_price)}" if not pd.isna(s2) else f"Deep Support ${int(zone3_price)}"
            elif confluence == 'BALANCED':
                zone1_price = daily_val if not pd.isna(daily_val) else d100
                zone1_label = f"Lower VA ${int(zone1_price)}" if not pd.isna(daily_val) else f"D100 ${int(d100)}"
                zone2_price = d200
                zone2_label = f"D200 ${int(d200)}"
                zone3_price = s1 if not pd.isna(s1) else d200 * 0.95
                zone3_label = f"S1 ${int(zone3_price)}" if not pd.isna(s1) else f"Near D200 ${int(zone3_price)}"
            else:  # WEAK
                zone1_price = d100
                zone1_label = f"D100 ${int(d100)}"
                zone2_price = d200
                zone2_label = f"D200 ${int(d200)}"
                zone3_price = s1 if not pd.isna(s1) else d200 * 0.95
                zone3_label = f"S1 ${int(zone3_price)}" if not pd.isna(s1) else f"Below D200 ${int(zone3_price)}"
            
            # Calculate shares for each zone
            zone1_shares_low = int(zone1_low / zone1_price)
            zone1_shares_high = int(zone1_high / zone1_price)
            zone2_shares_low = int(zone2_low / zone2_price)
            zone2_shares_high = int(zone2_high / zone2_price)
            zone3_shares_low = int(zone3_low / zone3_price)
            zone3_shares_high = int(zone3_high / zone3_price)
            
            entry_table_data = [
                ['Tranche', 'Capital', 'Target Zone', 'Est. Shares', 'Rationale'],
                ['Zone 1\nPrimary', f"${zone1_low:,.0f}-\n${zone1_high:,.0f}", zone1_label, f"{zone1_shares_low}-\n{zone1_shares_high}", 'Initial entry at\nkey support'],
                ['Zone 2\nSecondary', f"${zone2_low:,.0f}-\n${zone2_high:,.0f}", zone2_label, f"{zone2_shares_low}-\n{zone2_shares_high}", 'Average down\nif dips deeper'],
                ['Zone 3\nFinal', f"${zone3_low:,.0f}-\n${zone3_high:,.0f}", zone3_label, f"{zone3_shares_low}-\n{zone3_shares_high}", 'Capitulation\nbuy only']
            ]
            
            entry_table = Table(entry_table_data, colWidths=[1.0*inch, 1.2*inch, 1.3*inch, 1.0*inch, 1.5*inch])
            entry_table.setStyle(TableStyle([
                ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#3498db')),
                ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
                ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
                ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
                ('FONTSIZE', (0, 0), (-1, 0), 9),
                ('FONTSIZE', (0, 1), (-1, -1), 8),
                ('GRID', (0, 0), (-1, -1), 1, colors.grey),
                ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
            ]))
            section_elements.append(entry_table)
        else:
            section_elements.append(Paragraph("<i>Already at or above target allocation</i>", styles['Normal']))
        
        # Add completed section with KeepTogether
        elements.append(KeepTogether(section_elements))
        elements.append(Spacer(1, 0.15*inch))
        
        # === SECTION: Complete Technical Landscape ===
        section_elements = []
        section_elements.append(Paragraph("Complete Technical Landscape", section_style))
        
        tech_table_data = [
            ['Type', 'Level 1', 'Level 2', 'Level 3', 'Current'],
            ['Support', f"S1: ${int(s1)}" if not pd.isna(s1) else 'N/A', 
             f"S2: ${int(s2)}" if not pd.isna(s2) else 'N/A',
             f"S3: ${int(s3)}" if not pd.isna(s3) else 'N/A', ''],
            ['Daily SMAs', f"D20: ${int(d20)}", f"D50: ${int(d50)}", f"D100: ${int(d100)}", f"D200: ${int(d200)}"],
            ['Weekly SMAs', f"W10: ${int(w10)}", f"W20: ${int(w20)}", f"W200: ${int(w200)}", ''],
            ['Daily VP', f"Lower VA: ${int(daily_val)}" if not pd.isna(daily_val) else 'N/A',
             f"POC: ${int(daily_poc)}" if not pd.isna(daily_poc) else 'N/A',
             f"Upper VA: ${int(daily_vah)}" if not pd.isna(daily_vah) else 'N/A', ''],
            ['Weekly VP', f"Lower VA: ${int(weekly_val)}" if not pd.isna(weekly_val) else 'N/A',
             f"POC: ${int(weekly_poc)}" if not pd.isna(weekly_poc) else 'N/A',
             f"Upper VA: ${int(weekly_vah)}" if not pd.isna(weekly_vah) else 'N/A', ''],
            ['Resistance', f"R1: ${int(r1)}" if not pd.isna(r1) else 'N/A',
             f"R2: ${int(r2)}" if not pd.isna(r2) else 'N/A',
             f"R3: ${int(r3)}" if not pd.isna(r3) else 'N/A', f"Price: ${price:.2f}"]
        ]
        
        tech_table = Table(tech_table_data, colWidths=[1.3*inch, 1.2*inch, 1.2*inch, 1.2*inch, 1.1*inch])
        tech_table.setStyle(TableStyle([
            ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#2c3e50')),
            ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
            ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
            ('FONTSIZE', (0, 0), (-1, 0), 9),
            ('FONTSIZE', (0, 1), (-1, -1), 7),
            ('GRID', (0, 0), (-1, -1), 1, colors.grey),
        ]))
        section_elements.append(tech_table)
        
        elements.append(KeepTogether(section_elements))
        elements.append(Spacer(1, 0.15*inch))
        
        # === SECTION: Risk Management ===
        section_elements = []
        section_elements.append(Paragraph("Risk Management", section_style))
        
        # Suggested stop loss (below key support)
        if confluence == 'EXTENDED':
            stop_price = d200 * 0.97
            stop_reason = "3% below D200 (key long-term support)"
        elif confluence == 'BALANCED':
            stop_price = (daily_val if not pd.isna(daily_val) else d100) * 0.96
            stop_reason = "4% below Lower VA / D100"
        else:
            stop_price = d200 * 0.95
            stop_reason = "5% below D200 (structure broken)"
        
        # Max loss per zone (2% of portfolio)
        max_loss_per_zone = total_portfolio_value * 0.02
        
        # Handle R1 for risk/reward text
        r1_text = f"${int(r1)}" if not pd.isna(r1) else "key resistance"
        
        risk_text = f"""
        <b>Suggested Stop Loss:</b> ${stop_price:.2f} ({stop_reason})<br/>
        <b>Max Loss Per Entry:</b> ${max_loss_per_zone:,.0f} (2% of portfolio)<br/>
        <b>Position Sizing:</b> Scale in across 3 zones to average {target_pct}% allocation<br/>
        <b>Risk/Reward:</b> Targeting move to {r1_text} with stop at ${stop_price:.2f}
        """
        section_elements.append(Paragraph(risk_text, styles['Normal']))
        
        elements.append(KeepTogether(section_elements))
        elements.append(Spacer(1, 0.15*inch))
        
        # === SECTION: Basket Context ===
        section_elements = []
        section_elements.append(Paragraph("Basket Context & Conviction", section_style))
        
        # Find which baskets contain this ticker
        ticker_baskets = [(name, constituents) for name, constituents in baskets.items() if ticker in constituents]
        
        if ticker_baskets:
            basket_text = ""
            for basket_name, constituents in ticker_baskets:
                basket_signal = df[df['ticker'] == f'[{basket_name}]']['signal'].values
                basket_signal_str = basket_signal[0] if len(basket_signal) > 0 else 'UNKNOWN'
                
                if basket_signal_str == "FULL HOLD + ADD":
                    conviction = "HIGH - Basket bullish, sector strength confirmed"
                    color_note = "✅"
                elif "HOLD" in basket_signal_str:
                    conviction = "MODERATE - Basket neutral, selective adds OK"
                    color_note = "⚠️"
                else:
                    conviction = "LOW - Basket weak, use caution"
                    color_note = "🔴"
                
                basket_text += f"{color_note} <b>{basket_name}:</b> {basket_signal_str}<br/>"
                basket_text += f"<i>Conviction: {conviction}</i><br/>"
            section_elements.append(Paragraph(basket_text, styles['Normal']))
        else:
            section_elements.append(Paragraph("<i>Not in any basket - standalone ticker</i>", styles['Normal']))
        
        elements.append(KeepTogether(section_elements))
        elements.append(Spacer(1, 0.15*inch))
        
        # === SECTION: Confluence Explanation ===
        section_elements = []
        section_elements.append(Paragraph("Why This Confluence Rating?", section_style))
        
        if confluence == 'EXTENDED':
            explanation = f"""
            <b>EXTENDED</b> means price is significantly above key moving averages:<br/>
            • Current price ${price:.2f} is {((price/d200-1)*100):.1f}% above D200 (${int(d200)})<br/>
            • Technically stretched - wait for deeper pullback to avoid buying the top<br/>
            • <b>Strategy:</b> Patient accumulation only at D200/S1 levels (don't chase)
            """
        elif confluence == 'BALANCED':
            explanation = f"""
            <b>BALANCED</b> means price near healthy support with room to run:<br/>
            • Price ${price:.2f} within {abs((price/d100-1)*100):.1f}% of D100 (${int(d100)})<br/>
            • Technical setup favorable - not overextended, not broken<br/>
            • <b>Strategy:</b> Safe to enter on dips to Lower VA / D100 zones
            """
        else:  # WEAK
            explanation = f"""
            <b>WEAK</b> means price below key support, trend questionable:<br/>
            • Price ${price:.2f} is {((1-price/d100)*100):.1f}% below D100 (${int(d100)})<br/>
            • May need base-building - extra caution on position size<br/>
            • <b>Strategy:</b> Small initial buys, scale aggressively only on turnaround
            """
        section_elements.append(Paragraph(explanation, styles['Normal']))
        
        elements.append(KeepTogether(section_elements))
        
        # Page break before next ticker (except last one)
        if idx < len(eligible) - 1:
            elements.append(PageBreak())
    
    # Build PDF
    doc.build(elements)
    print(f"✅ Buy Detailed PDF exported: {pdf_path}")
    print(f"   {len(eligible)} tickers analyzed (alphabetically sorted)")

# Generate the detailed buy PDF
export_buy_detailed_to_pdf(df)

✅ Buy Detailed PDF exported: C:\Users\karms\Downloads\buy_detailed_20260105_1758.pdf
   5 tickers analyzed (alphabetically sorted)


In [ ]:
# ===================================================================================================
# CELL 7: EXPORT SELL SUMMARY PDF (Quick Reference)
# ===================================================================================================
# Purpose: Generate single-page table of positions requiring reduction
# Inputs:  df (from Cell 2), holdings.csv
# Outputs: sell_summary_YYYYMMDD_HHMM.pdf in ~/Downloads
# Format:  Compact table with reduction %, tranches, zones, keep amounts
# Use:     Quick defensive action review - which positions to trim and at what prices
# Note:    Only generates if defensive signals detected AND positions held
# ===================================================================================================

from technical_analysis import calculate_reduction_amounts, get_reduction_tranches
from datetime import datetime, timedelta

def export_sell_summary_to_pdf(df, starting_cash=104967):
    """Generate a PDF report for capital protection - positions requiring reductions"""
    
    # Load current holdings
    holdings_file = ROOT / 'holdings.csv'
    holdings_dict = {}
    if holdings_file.exists():
        holdings_df = pd.read_csv(holdings_file)
        for _, row in holdings_df.iterrows():
            holdings_dict[row['ticker']] = {
                'shares': row['shares'],
                'avg_cost': row['avg_cost']
            }
    else:
        print("⚠️  holdings.csv not found - cannot generate sell summary")
        return
    
    # Calculate total portfolio value
    total_holdings_value = 0
    for ticker, holding in holdings_dict.items():
        if holding['shares'] > 0:
            current_price = df[df['ticker'] == ticker]['current_price'].values
            if len(current_price) > 0:
                total_holdings_value += holding['shares'] * current_price[0]
    
    total_portfolio_value = total_holdings_value + starting_cash
    
    # Filter for positions requiring capital protection
    defensive_signals = ["HOLD MOST → REDUCE", "REDUCE", "LIGHT / CASH", "CASH", "FULL CASH / DEFEND"]
    positions_to_reduce = df[
        (df['signal'].isin(defensive_signals)) & 
        (~df['ticker'].str.startswith('['))  # Exclude baskets
    ]
    
    # Only generate if there are positions to protect
    if positions_to_reduce.empty:
        print("✅ No defensive signals - capital protection not required")
        return
    
    # Calculate next trading day
    today = datetime.now()
    next_trading_day = today + timedelta(days=1)
    while next_trading_day.weekday() >= 5:
        next_trading_day += timedelta(days=1)
    
    # Generate PDF filename with unified timestamp - save to workspace
    pdf_filename = f"sell_summary_{TIMESTAMP}.pdf"
    pdf_path = RESULTS_DIR / pdf_filename
    
    # Create PDF
    doc = SimpleDocTemplate(str(pdf_path), pagesize=letter,
                           rightMargin=30, leftMargin=30,
                           topMargin=30, bottomMargin=18)
    
    elements = []
    styles = getSampleStyleSheet()
    
    # Title style with red/warning theme
    title_style = ParagraphStyle(
        'DefensiveTitle',
        parent=styles['Heading1'],
        fontSize=18,
        textColor=colors.HexColor('#c0392b'),  # Red for defensive
        spaceAfter=20,
        alignment=TA_CENTER
    )
    
    # Add title
    title_text = f"⚠️ Capital Protection Summary – For {next_trading_day.strftime('%A, %B %d, %Y')}<br/><font size=12>(Defensive Actions Required)</font>"
    elements.append(Paragraph(title_text, title_style))
    elements.append(Spacer(1, 0.2*inch))
    
    # Add warning summary
    warning_text = f"""
    <b>Capital Protection Triggered</b> — Defensive signals detected:<br/>
    <br/>
    <b>Protection Philosophy:</b><br/>
    • <b>Asymmetric Defense:</b> Quick to protect on bearish turns, patient to re-enter<br/>
    • <b>Phased Exits:</b> Gradual reductions into strength/rallies (never panic sell at lows)<br/>
    • <b>Mechanical Rules:</b> No emotion — based on dual-timeframe Larsson state<br/>
    <br/>
    <b>Portfolio Status:</b><br/>
    <b>Total Portfolio Value:</b> ${total_portfolio_value:,.0f}<br/>
    <b>Current Holdings:</b> ${total_holdings_value:,.0f}<br/>
    <b>Cash Available:</b> ${starting_cash:,.0f}<br/>
    <br/>
    <b>Proceed amounts go to cash</b> — ready to redeploy only into FULL HOLD + ADD names when conditions improve.
    """
    elements.append(Paragraph(warning_text, styles['Normal']))
    elements.append(Spacer(1, 0.3*inch))
    
    # Prepare table data
    table_data = [['Ticker', 'Signal', 'Current\nValue', 'Reduce\n%', 'Tranche 1', 'Zone 1', 'Tranche 2', 'Zone 2', 'Keep']]
    
    for _, row in positions_to_reduce.iterrows():
        ticker = row['ticker']
        signal = row['signal']
        price = row['current_price']
        
        # Get holding
        holding = holdings_dict.get(ticker, {'shares': 0, 'avg_cost': 0})
        if holding['shares'] == 0:
            continue  # Skip if no position
        
        current_value = holding['shares'] * price
        
        # Calculate reduction amounts
        reduction_amount, keep_amount, reduction_pct = calculate_reduction_amounts(signal, current_value)
        
        # Get tranches
        tranches = get_reduction_tranches(signal, reduction_pct, reduction_amount)
        
        # Get resistance levels for sell zones
        r1 = row['r1'] if not pd.isna(row['r1']) else price * 1.02
        r2 = row['r2'] if not pd.isna(row['r2']) else price * 1.05
        r3 = row['r3'] if not pd.isna(row['r3']) else price * 1.08
        
        # Format tranches for display
        if len(tranches) == 1:
            # Single tranche
            tranche1_amount, tranche1_pct, timing1 = tranches[0]
            tranche1_shares = holding['shares'] * tranche1_pct
            # Show decimals for fractional shares
            if tranche1_shares < 1:
                shares1_str = f"{tranche1_shares:.2f}"
            else:
                shares1_str = f"{tranche1_shares:.1f}"
            
            zone1 = f"Target: ${int(r1)}+\n(current ${int(price)})"
            
            table_data.append([
                ticker,
                signal.replace("HOLD MOST → REDUCE", "HOLD MOST\n→ REDUCE"),
                f"${current_value:,.0f}",
                f"{int(reduction_pct*100)}%",
                f"${tranche1_amount:,.0f}\n({shares1_str})",
                zone1,
                "—",
                "—",
                f"${keep_amount:,.0f}"
            ])
        
        elif len(tranches) == 2:
            # Two tranches
            tranche1_amount, tranche1_pct, timing1 = tranches[0]
            tranche2_amount, tranche2_pct, timing2 = tranches[1]
            tranche1_shares = holding['shares'] * tranche1_pct
            tranche2_shares = holding['shares'] * tranche2_pct
            
            # Show decimals for fractional shares
            if tranche1_shares < 1:
                shares1_str = f"{tranche1_shares:.2f}"
            else:
                shares1_str = f"{tranche1_shares:.1f}"
            if tranche2_shares < 1:
                shares2_str = f"{tranche2_shares:.2f}"
            else:
                shares2_str = f"{tranche2_shares:.1f}"
            
            zone1 = f"Target: ${int(r1)}\n(current ${int(price)})"
            zone2 = f"Target: ${int(r2)}+\n(wait for rally)"
            
            table_data.append([
                ticker,
                signal.replace("HOLD MOST → REDUCE", "HOLD MOST\n→ REDUCE"),
                f"${current_value:,.0f}",
                f"{int(reduction_pct*100)}%",
                f"${tranche1_amount:,.0f}\n({shares1_str})",
                zone1,
                f"${tranche2_amount:,.0f}\n({shares2_str})",
                zone2,
                f"${keep_amount:,.0f}"
            ])
        
        else:  # Three tranches (FULL CASH / DEFEND)
            tranche1_amount, tranche1_pct, timing1 = tranches[0]
            tranche2_amount, tranche2_pct, timing2 = tranches[1]
            tranche3_amount, tranche3_pct, timing3 = tranches[2]
            tranche1_shares = holding['shares'] * tranche1_pct
            tranche2_shares = holding['shares'] * tranche2_pct
            tranche3_shares = holding['shares'] * tranche3_pct
            
            # Show decimals for fractional shares
            if tranche1_shares < 1:
                shares1_str = f"{tranche1_shares:.2f}"
            else:
                shares1_str = f"{tranche1_shares:.1f}"
            
            zone1 = f"Target: ${int(r1)}\n(current ${int(price)})"
            zone2 = f"Target: ${int(r2)}+\nup to R3 ${int(r3)}"
            
            # Combine tranche 2 and 3 for display
            combined_t2_amount = tranche2_amount + tranche3_amount
            combined_t2_shares = tranche2_shares + tranche3_shares
            if combined_t2_shares < 1:
                shares2_str = f"{combined_t2_shares:.2f}"
            else:
                shares2_str = f"{combined_t2_shares:.1f}"
            
            table_data.append([
                ticker,
                signal.replace("FULL CASH / DEFEND", "FULL CASH\n/ DEFEND"),
                f"${current_value:,.0f}",
                f"{int(reduction_pct*100)}%",
                f"${tranche1_amount:,.0f}\n({shares1_str})",
                zone1,
                f"${combined_t2_amount:,.0f}\n({shares2_str})",
                zone2,
                f"${keep_amount:,.0f}"
            ])
    
    if len(table_data) == 1:  # Only header
        print("✅ No current holdings require reduction")
        return
    
    # Create table
    table = Table(table_data, colWidths=[0.7*inch, 0.9*inch, 0.75*inch, 0.5*inch, 0.85*inch, 0.85*inch, 0.85*inch, 0.85*inch, 0.7*inch])
    
    # Apply table style with warning color scheme
    style_commands = [
        ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#c0392b')),  # Red header
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 8),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.white),
        ('GRID', (0, 0), (-1, -1), 1, colors.grey),
        ('FONTSIZE', (0, 1), (-1, -1), 7),
        ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
    ]
    
    # Add signal-based row coloring
    for i in range(1, len(table_data)):
        signal_text = table_data[i][1]
        if "FULL CASH" in signal_text:
            color = colors.HexColor('#f8d7da')  # Light red
        elif "CASH" in signal_text or "REDUCE" in signal_text:
            color = colors.HexColor('#fff3cd')  # Light yellow/warning
        else:
            color = colors.HexColor('#d1ecf1')  # Light blue (lightest warning)
        style_commands.append(('BACKGROUND', (0, i), (-1, i), color))
    
    table.setStyle(TableStyle(style_commands))
    
    elements.append(table)
    elements.append(Spacer(1, 0.3*inch))
    
    # Add execution guidance
    exec_text = """
    <b>Execution Plan</b><br/>
    <br/>
    <b>Zone Prices Explained:</b> Target prices are RESISTANCE levels to sell into (above current). Wait for bounces to these levels for better exit prices. "Current" means you can sell now if urgent, but waiting captures better prices.<br/>
    <br/>
    • <b>Tranche 1:</b> Sell at Target price (preferred) or current market (if urgent)<br/>
    • <b>Tranche 2:</b> Wait patiently for rally to Target resistance — sell into strength, not weakness<br/>
    • <b>Never panic sell at lows</b> — gradual exits into rallies preserve capital and achieve better prices<br/>
    • <b>Proceeds to cash</b> — do not redeploy until signals improve to FULL HOLD + ADD<br/>
    <br/>
    <b>Re-Entry Criteria:</b> Only after weekly Larsson state reclaims bullish with daily confirmation.
    """
    elements.append(Paragraph(exec_text, styles['Normal']))
    elements.append(Spacer(1, 0.2*inch))
    
    # Add footer
    footer_text = f"<font size=8>Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}</font>"
    elements.append(Paragraph(footer_text, styles['Normal']))
    
    # Build PDF
    doc.build(elements)
    print(f"⚠️  Sell Summary PDF saved to Downloads: {pdf_path}")

# Execute sell summary PDF generation
export_sell_summary_to_pdf(df, starting_cash=104967)

⚠️  Sell Summary PDF saved to Downloads: C:\Users\karms\Downloads\sell_summary_20260105_1758.pdf


In [ ]:
# ===================================================================================================
# CELL 7: EXPORT SELL DETAILED PDF (Defensive Exit Playbook)
# ===================================================================================================
# Purpose: Generate multi-page detailed exit plan - ONE PAGE PER POSITION
# Inputs:  df (from Cell 2), holdings.csv
# Outputs: sell_detailed_YYYYMMDD_HHMM.pdf in ~/Downloads
# Format:  Cover page + 1 page per position with:
#          - P&L context (cost basis, unrealized gains/losses)
#          - All 3 tranches shown separately (not combined)
#          - 4 exit scenarios (rally to R2, drop to S1, break R3, extended weakness)
#          - Complete resistance map (R1/R2/R3, Upper VA, POC)
#          - Detailed re-entry criteria (5 conditions that must ALL be met)
#          - What triggered the defensive signal
# Use:     Complete exit playbook for capital protection decisions
# ===================================================================================================

from datetime import datetime, timedelta
from reportlab.platypus import KeepTogether, PageBreak

def export_sell_detailed_to_pdf(df, starting_cash=104967):
    """Generate detailed multi-page PDF with one page per position requiring reduction"""
    
    # Load holdings
    holdings_file = ROOT / 'holdings.csv'
    holdings_dict = {}
    if holdings_file.exists():
        holdings_df = pd.read_csv(holdings_file)
        for _, row in holdings_df.iterrows():
            holdings_dict[row['ticker']] = {'shares': row['shares'], 'avg_cost': row['avg_cost']}
    else:
        print("⚠️  holdings.csv not found - cannot generate sell detailed PDF")
        return
    
    # Calculate portfolio value
    total_holdings_value = 0
    for ticker, holding in holdings_dict.items():
        if holding['shares'] > 0:
            current_price = df[df['ticker'] == ticker]['current_price'].values
            if len(current_price) > 0:
                total_holdings_value += holding['shares'] * current_price[0]
    
    total_portfolio_value = total_holdings_value + starting_cash
    
    # Filter for positions requiring capital protection
    defensive_signals = ["HOLD MOST → REDUCE", "REDUCE", "LIGHT / CASH", "CASH", "FULL CASH / DEFEND"]
    positions_to_reduce = df[
        (df['signal'].isin(defensive_signals)) & 
        (~df['ticker'].str.startswith('['))
    ]
    
    # Only positions with actual holdings, sorted alphabetically
    positions_with_holdings = []
    for _, row in positions_to_reduce.iterrows():
        ticker = row['ticker']
        holding = holdings_dict.get(ticker, {'shares': 0, 'avg_cost': 0})
        if holding['shares'] > 0:
            positions_with_holdings.append(row)
    
    # Sort alphabetically by ticker
    positions_with_holdings = sorted(positions_with_holdings, key=lambda r: r['ticker'])
    
    if not positions_with_holdings:
        print("✅ No positions require reduction - skipping sell detailed PDF")
        return
    
    # Generate PDF
    today = datetime.now()
    next_trading_day = today + timedelta(days=1)
    while next_trading_day.weekday() >= 5:
        next_trading_day += timedelta(days=1)
    
    pdf_filename = f"sell_detailed_{TIMESTAMP}.pdf"
    pdf_path = RESULTS_DIR / pdf_filename
    
    doc = SimpleDocTemplate(str(pdf_path), pagesize=letter,
                           rightMargin=30, leftMargin=30,
                           topMargin=30, bottomMargin=30)
    
    elements = []
    styles = getSampleStyleSheet()
    
    # Custom styles
    ticker_title_style = ParagraphStyle(
        'TickerTitle',
        parent=styles['Heading1'],
        fontSize=24,
        textColor=colors.HexColor('#c0392b'),  # Red for defensive
        spaceAfter=10,
        alignment=TA_CENTER
    )
    
    section_style = ParagraphStyle(
        'Section',
        parent=styles['Heading2'],
        fontSize=14,
        textColor=colors.HexColor('#c0392b'),
        spaceAfter=8,
        spaceBefore=12
    )
    
    warning_style = ParagraphStyle(
        'Warning',
        parent=styles['Normal'],
        textColor=colors.HexColor('#e74c3c'),
        fontSize=10
    )
    
    # Cover page
    cover_title = f"⚠️ Capital Protection - Detailed Exit Playbook<br/><font size=14>For {next_trading_day.strftime('%A, %B %d, %Y')}</font>"
    elements.append(Paragraph(cover_title, ticker_title_style))
    elements.append(Spacer(1, 0.3*inch))
    
    summary = f"""
    <b>Portfolio Context:</b><br/>
    Total Portfolio Value: ${total_portfolio_value:,.0f}<br/>
    Current Holdings: ${total_holdings_value:,.0f}<br/>
    Cash Available: ${starting_cash:,.0f}<br/>
    <br/>
    <b>Positions Requiring Reduction: {len(positions_with_holdings)}</b><br/>
    {', '.join([row['ticker'] for row in positions_with_holdings])}<br/>
    <br/>
    <b>Defense Philosophy:</b> Asymmetric protection — quick to defend on bearish turns,
    patient to re-enter. Gradual exits into rallies, never panic sell at lows.
    Proceeds to cash until signals improve to FULL HOLD + ADD.<br/>
    <br/>
    <i>Each position analyzed on separate page with complete tranche breakdown,
    exit scenarios, re-entry criteria, and P&L context.</i>
    """
    elements.append(Paragraph(summary, styles['Normal']))
    elements.append(PageBreak())
    
    # One page per position
    for idx, row in enumerate(positions_with_holdings):
        ticker = row['ticker']
        signal = row['signal']
        price = row['current_price']
        
        # Get holding and P&L
        holding = holdings_dict[ticker]
        shares = holding['shares']
        avg_cost = holding['avg_cost']
        position_value = shares * price
        total_gain_loss = position_value - (shares * avg_cost)
        gain_loss_pct = ((price / avg_cost) - 1) * 100 if avg_cost > 0 else 0
        
        # Technical levels
        r1, r2, r3 = row['r1'], row['r2'], row['r3']
        s1, s2, s3 = row['s1'], row['s2'], row['s3']
        daily_poc, daily_vah, daily_val = row['daily_poc'], row['daily_vah'], row['daily_val']
        d100, d200 = row['d100'], row['d200']
        
        # Calculate reduction
        reduction_amount, keep_amount, reduction_pct = calculate_reduction_amounts(signal, position_value)
        tranches = get_reduction_tranches(signal, reduction_pct, reduction_amount)
        
        # === TICKER HEADER ===
        elements.append(Paragraph(f"⚠️ {ticker}", ticker_title_style))
        elements.append(Paragraph(f"<b>Signal:</b> {signal}", warning_style))
        elements.append(Spacer(1, 0.15*inch))
        
        # === SECTION: Position & P/L Context ===
        section_elements = []
        section_elements.append(Paragraph("Position & P/L Context", section_style))
        
        gain_loss_text = f"+${total_gain_loss:,.0f} (+{gain_loss_pct:.1f}%)" if total_gain_loss > 0 else f"-${abs(total_gain_loss):,.0f} ({gain_loss_pct:.1f}%)"
        gain_loss_color = "green" if total_gain_loss > 0 else "red"
        
        # Determine unit type (coin vs share)
        unit_type = "per coin" if ticker in ['BTC-USD', 'SOL-USD'] else "per share"
        
        position_text = f"""
        <b>Current Position:</b> {shares:.4f} shares @ ${price:,.2f} = ${position_value:,.0f}<br/>
        <b>Cost Basis:</b> ${int(avg_cost):,} {unit_type}<br/>
        <b>Unrealized P/L:</b> <font color="{gain_loss_color}">{gain_loss_text}</font><br/>
        <br/>
        <b>Why Protect?</b> {get_signal_description(signal)}<br/>
        <b>Keep Amount:</b> ${keep_amount:,.0f} ({int((1-reduction_pct)*100)}% of position)
        """
        section_elements.append(Paragraph(position_text, styles['Normal']))
        
        elements.append(KeepTogether(section_elements))
        elements.append(Spacer(1, 0.15*inch))
        
        # === SECTION: Complete Tranche Breakdown ===
        section_elements = []
        section_elements.append(Paragraph("Complete Tranche Breakdown", section_style))
        
        # Show all tranches separately (not combined)
        tranche_table_data = [['Tranche', 'Amount', 'Shares', 'Target Zone', 'Trigger']]
        
        # Handle resistance levels with fallbacks
        r1_price = r1 if not pd.isna(r1) else price * 1.02
        r2_price = r2 if not pd.isna(r2) else price * 1.05
        r3_price = r3 if not pd.isna(r3) else price * 1.08
        
        for i, (amount, pct, timing) in enumerate(tranches, 1):
            tranche_shares = shares * pct
            shares_str = f"{tranche_shares:.2f}" if tranche_shares < 1 else f"{tranche_shares:.1f}"
            
            # Determine zone and trigger
            if i == 1:
                zone = f"R1 ${int(r1_price):,}+"
                trigger = f"Limit @ ${int(r1_price):,} or market"
            elif i == 2:
                zone = f"R2 ${int(r2_price):,}+"
                trigger = f"Sell into rally @ ${int(r2_price):,}+"
            else:  # Tranche 3
                zone = f"R3 ${int(r3_price):,}"
                trigger = f"Exit on rally to ${int(r3_price):,}"
            
            tranche_table_data.append([
                f"T{i}\n{timing}",
                f"${amount:,.0f}\n({int(pct*100)}%)",
                shares_str,
                zone,
                trigger
            ])
        
        tranche_table = Table(tranche_table_data, colWidths=[1.3*inch, 1.1*inch, 0.7*inch, 1.2*inch, 2.2*inch])
        tranche_table.setStyle(TableStyle([
            ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#c0392b')),
            ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
            ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
            ('FONTSIZE', (0, 0), (-1, 0), 9),
            ('FONTSIZE', (0, 1), (-1, -1), 7),
            ('GRID', (0, 0), (-1, -1), 1, colors.grey),
            ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
        ]))
        section_elements.append(tranche_table)
        
        elements.append(KeepTogether(section_elements))
        elements.append(Spacer(1, 0.15*inch))
        
        # === SECTION: Exit Scenarios ===
        section_elements = []
        section_elements.append(Paragraph("Multiple Exit Scenarios", section_style))
        
        # Calculate S1 price with fallback
        s1_price = int(s1) if not pd.isna(s1) else int(price*0.95)
        
        scenarios_text = f"""
        <b>Scenario A: Price Rallies to R2 First</b><br/>
        • Execute Tranche 2 at ${int(r2_price):,}+ (sell into strength)<br/>
        • Hold Tranche 1 for any R1 bounce (${int(r1_price):,})<br/>
        • Reserve Tranche 3 for full rally to R3 (${int(r3_price):,})<br/>
        <br/>
        <b>Scenario B: Price Drops to S1 (${s1_price:,})</b><br/>
        • Do NOT panic sell at lows<br/>
        • Wait patiently for dead cat bounce to R1 (${int(r1_price):,})<br/>
        • Execute Tranche 1 on bounce, hold rest for higher prices<br/>
        <br/>
        <b>Scenario C: Price Breaks Above R3 (${int(r3_price):,})</b><br/>
        • Signal may be improving - reconsider reduction<br/>
        • Check if weekly/daily Larsson states are reclaiming bullish<br/>
        • If trend reverses back to bullish, cancel remaining tranches<br/>
        <br/>
        <b>Scenario D: Extended Weakness Below S2</b><br/>
        • Accept defensive posture - do not average down<br/>
        • Execute all tranches on any bounce to resistance<br/>
        • Preserve capital for future opportunities
        """
        section_elements.append(Paragraph(scenarios_text, styles['Normal']))
        
        elements.append(KeepTogether(section_elements))
        elements.append(Spacer(1, 0.15*inch))
        
        # === SECTION: Complete Resistance Map ===
        section_elements = []
        section_elements.append(Paragraph("Complete Resistance Map", section_style))
        
        resistance_table_data = [
            ['Level', 'Price', 'Type', 'Exit Strategy'],
            ['R1', f"${int(r1_price):,}" if not pd.isna(r1) else f"${int(r1_price):,} (est)", 
             'Minor Resistance', 'First bounce target - Tranche 1'],
            ['R2', f"${int(r2_price):,}" if not pd.isna(r2) else f"${int(r2_price):,} (est)", 
             'Key Resistance', 'Strong rally target - Tranche 2'],
            ['R3', f"${int(r3_price):,}" if not pd.isna(r3) else f"${int(r3_price):,} (est)", 
             'Major Overhead', 'Full rally target - Tranche 3'],
            ['Upper VA', f"${int(daily_vah):,}" if not pd.isna(daily_vah) else 'N/A', 
             'Volume Resistance', 'Heavy supply zone'],
            ['POC', f"${int(daily_poc):,}" if not pd.isna(daily_poc) else 'N/A', 
             'Volume Node', 'High activity price'],
            ['Current', f"${price:,.2f}", 'Market Price', 'Your decision point']
        ]
        
        resistance_table = Table(resistance_table_data, colWidths=[1.0*inch, 1.3*inch, 1.5*inch, 2.2*inch])
        resistance_table.setStyle(TableStyle([
            ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#2c3e50')),
            ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
            ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
            ('FONTSIZE', (0, 0), (-1, 0), 9),
            ('FONTSIZE', (0, 1), (-1, -1), 8),
            ('GRID', (0, 0), (-1, -1), 1, colors.grey),
            ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
            ('BACKGROUND', (0, -1), (-1, -1), colors.lightyellow),  # Highlight current price
        ]))
        section_elements.append(resistance_table)
        
        elements.append(KeepTogether(section_elements))
        elements.append(Spacer(1, 0.15*inch))
        
        # === SECTION: Re-Entry Criteria (Detailed) ===
        section_elements = []
        section_elements.append(Paragraph("Complete Re-Entry Criteria", section_style))
        
        reentry_text = f"""
        <b>DO NOT re-enter until ALL conditions are met:</b><br/>
        <br/>
        <b>Condition 1: Weekly Larsson State</b><br/>
        • Weekly must reclaim +1 (bullish) from current state<br/>
        • Must hold +1 for at least 2 consecutive weeks (no whipsaw)<br/>
        <br/>
        <b>Condition 2: Daily Larsson State</b><br/>
        • Daily must also be +1 (bullish confirmation)<br/>
        • Must maintain +1 for at least 3-5 consecutive days<br/>
        <br/>
        <b>Condition 3: Price Structure</b><br/>
        • Price must reclaim and hold above D100 (${int(d100):,})<br/>
        • Ideally reclaim D200 (${int(d200):,}) for strong trend confirmation<br/>
        • Hold above reclaimed level for 5+ trading days<br/>
        <br/>
        <b>Condition 4: Confluence Improvement</b><br/>
        • Confluence must improve from WEAK → BALANCED or BALANCED → EXTENDED<br/>
        • Volume Profile support (price holds above Lower VA)<br/>
        <br/>
        <b>Condition 5: Signal Strength</b><br/>
        • Signal must flip back to "FULL HOLD + ADD"<br/>
        • Not just "HOLD" - need full conviction before re-entering<br/>
        <br/>
        <b>■ Re-Entry Timing:</b> Wait for ALL 5 conditions. Patience prevents catching falling knives.
        """
        section_elements.append(Paragraph(reentry_text, styles['Normal']))
        
        elements.append(KeepTogether(section_elements))
        elements.append(Spacer(1, 0.15*inch))
        
        # === SECTION: What Triggered This Defense ===
        section_elements = []
        section_elements.append(Paragraph("What Triggered Capital Protection?", section_style))
        
        trigger_text = f"""
        <b>Signal: {signal}</b><br/>
        <br/>
        <b>Technical Breakdown:</b><br/>
        • {get_signal_description(signal)}<br/>
        <br/>
        <b>Current Technical State:</b><br/>
        • Price: ${price:,.2f} (vs D100 ${int(d100):,}, D200 ${int(d200):,})<br/>
        • Confluence: {row['confluence']}<br/>
        • Recommendation: {row['recommendation']}<br/>
        <br/>
        <b>Why Asymmetric Defense?</b><br/>
        It's easier to re-enter on confirmed bullish turn than to recover from extended drawdown.
        Protecting capital now preserves buying power for better opportunities. This is mechanical,
        not emotional — follow the plan.
        """
        section_elements.append(Paragraph(trigger_text, styles['Normal']))
        
        elements.append(KeepTogether(section_elements))
        
        # Add page break except for last position
        if idx < len(positions_with_holdings) - 1:
            elements.append(PageBreak())
    
    # Build PDF
    doc.build(elements)
    print(f"📕 Sell Detailed PDF saved to Downloads: {pdf_path}")
    print(f"   {len(positions_with_holdings)} position(s) analyzed (alphabetically sorted)")

export_sell_detailed_to_pdf(df, starting_cash=104967)
# Execute detailed sell PDF generation

📕 Sell Detailed PDF saved to Downloads: C:\Users\karms\Downloads\sell_detailed_20260105_1758.pdf
   1 position(s) analyzed (alphabetically sorted)


In [10]:
# ===================================================================================================
# CELL 9: EXPORT TRADING ACTIONS TO EXCEL (3-Sheet Workbook)
# ===================================================================================================
# Purpose: Generate comprehensive Excel file with buy/sell actions + full technical data
# Inputs:  df (from Cell 2), holdings.csv, targets.csv
# Outputs: trading_actions_YYYYMMDD_HHMM.xlsx in ~/Downloads
# Sheets:  1) Buy Actions - FULL HOLD + ADD tickers with gaps, zones, share estimates
#          2) Sell Actions - Defensive signals with tranches, zones, keep amounts
#          3) Technical Data - Complete raw dataframe dump for reference
# Format:  Auto-adjusted column widths, frozen headers (row 1)
# Use:     Spreadsheet for tracking, sorting, filtering trading actions
# ===================================================================================================

from datetime import datetime
import pandas as pd

def export_trading_actions_to_excel(df, starting_cash=104967):
    """Export actionable trading data to 3-sheet Excel file"""
    
    # Generate filename with unified timestamp\n
    output_file = RESULTS_DIR / f'trading_actions_{TIMESTAMP}.xlsx'
    
    # Load holdings and targets for enrichment
    holdings_file = ROOT / 'holdings.csv'
    targets_file = ROOT / 'targets.csv'
    
    holdings_dict = {}
    if holdings_file.exists():
        holdings_df = pd.read_csv(holdings_file)
        for _, row in holdings_df.iterrows():
            holdings_dict[row['ticker']] = {'shares': row['shares'], 'avg_cost': row['avg_cost']}
    
    target_dict = {}
    if targets_file.exists():
        targets_df = pd.read_csv(targets_file)
        target_dict = dict(zip(targets_df['ticker'], targets_df['target_pct']))
    
    # Calculate total portfolio value
    total_holdings_value = 0
    for ticker, holding in holdings_dict.items():
        if holding['shares'] > 0:
            current_price = df[df['ticker'] == ticker]['current_price'].values
            if len(current_price) > 0:
                total_holdings_value += holding['shares'] * current_price[0]
    total_portfolio_value = total_holdings_value + starting_cash
    
    # === SHEET 1: BUY ACTIONS ===
    buy_signals = df[(df['signal'] == 'FULL HOLD + ADD') & (~df['ticker'].str.startswith('['))].copy()
    
    if not buy_signals.empty:
        buy_actions = []
        for _, row in buy_signals.iterrows():
            ticker = row['ticker']
            target_pct = target_dict.get(ticker, 0)
            target_val = (target_pct / 100) * total_portfolio_value if target_pct > 0 else 0
            
            # Current allocation
            holding = holdings_dict.get(ticker, {'shares': 0, 'avg_cost': 0})
            current_val = holding['shares'] * row['current_price']
            current_pct = (current_val / total_portfolio_value * 100) if total_portfolio_value > 0 else 0
            gap = target_val - current_val
            
            # Next add (40-50% of gap)
            remaining_gap = max(0, gap)
            next_add_low = remaining_gap * 0.4
            next_add_high = remaining_gap * 0.5
            next_add = f"${next_add_low:,.0f}-${next_add_high:,.0f}" if remaining_gap > 0 else "At Target"
            
            # Primary zone
            val = row['daily_val']
            poc = row['daily_poc']
            d100 = row['d100']
            d200 = row['d200']
            
            if row['confluence'] == 'EXTENDED':
                zone = f"D200 ${int(d200)} or D100 ${int(d100)}"
                zone_price = d200
            elif row['confluence'] == 'BALANCED' and not pd.isna(val):
                zone = f"Lower VA ${int(val)} or D100 ${int(d100)}"
                zone_price = val
            else:
                zone = f"D100 ${int(d100)}"
                zone_price = d100
            
            # Shares estimate
            shares_est = int((next_add_low + next_add_high) / 2 / zone_price) if remaining_gap > 0 else 0
            
            buy_actions.append({
                'Ticker': ticker,
                'Current Price': f"${row['current_price']:.2f}",
                'Signal': row['signal'],
                'Confluence': row['confluence'],
                'Recommendation': row['recommendation'],
                'Target %': f"{target_pct}%" if target_pct > 0 else 'N/A',
                'Current %': f"{current_pct:.1f}%",
                'Gap $': f"${gap:,.0f}" if gap > 0 else 'At/Above Target',
                'Next Add $': next_add,
                'Primary Zone': zone,
                'Est. Shares': shares_est if shares_est > 0 else 'N/A'
            })
        
        sheet1 = pd.DataFrame(buy_actions)
    else:
        sheet1 = pd.DataFrame([{'Ticker': 'No Buy Actions', 'Signal': 'No FULL HOLD + ADD signals'}])
    
    # === SHEET 2: SELL ACTIONS ===
    defensive_signals = ["HOLD MOST → REDUCE", "REDUCE", "LIGHT / CASH", "CASH", "FULL CASH / DEFEND"]
    sell_signals = df[(df['signal'].isin(defensive_signals)) & (~df['ticker'].str.startswith('['))].copy()
    
    if not sell_signals.empty:
        sell_actions = []
        for _, row in sell_signals.iterrows():
            ticker = row['ticker']
            holding = holdings_dict.get(ticker, {'shares': 0, 'avg_cost': 0})
            
            if holding['shares'] == 0:
                continue  # Skip if no position
            
            position_value = holding['shares'] * row['current_price']
            
            # Calculate reduction
            reduction_amount, keep_amount, reduction_pct = calculate_reduction_amounts(row['signal'], position_value)
            tranches = get_reduction_tranches(row['signal'], reduction_pct, reduction_amount)
            
            # Get zones
            r1 = row['r1'] if not pd.isna(row['r1']) else row['current_price'] * 1.02
            r2 = row['r2'] if not pd.isna(row['r2']) else row['current_price'] * 1.05
            
            # Format tranches
            if len(tranches) == 1:
                t1_amt, t1_pct, _ = tranches[0]
                t1_shares = holding['shares'] * t1_pct
                tranche1 = f"${t1_amt:,.0f} ({t1_shares:.2f} sh)" if t1_shares < 1 else f"${t1_amt:,.0f} ({t1_shares:.1f} sh)"
                zone1 = f"${int(r1)}+ (or current ${int(row['current_price'])})"
                tranche2 = '—'
                zone2 = '—'
            else:
                t1_amt, t1_pct, _ = tranches[0]
                t2_amt, t2_pct, _ = tranches[1]
                t1_shares = holding['shares'] * t1_pct
                t2_shares = holding['shares'] * t2_pct
                
                if len(tranches) == 3:
                    t3_amt, t3_pct, _ = tranches[2]
                    t2_amt += t3_amt
                    t2_shares += holding['shares'] * t3_pct
                
                tranche1 = f"${t1_amt:,.0f} ({t1_shares:.2f} sh)" if t1_shares < 1 else f"${t1_amt:,.0f} ({t1_shares:.1f} sh)"
                tranche2 = f"${t2_amt:,.0f} ({t2_shares:.2f} sh)" if t2_shares < 1 else f"${t2_amt:,.0f} ({t2_shares:.1f} sh)"
                zone1 = f"${int(r1)} (current ${int(row['current_price'])})"
                zone2 = f"${int(r2)}+ (wait for rally)"
            
            keep_pct = (keep_amount / position_value * 100) if position_value > 0 else 0
            
            sell_actions.append({
                'Ticker': ticker,
                'Current Price': f"${row['current_price']:.2f}",
                'Signal': row['signal'],
                'Position Value': f"${position_value:,.0f}",
                'Reduce %': f"{int(reduction_pct*100)}%",
                'Tranche 1': tranche1,
                'Zone 1': zone1,
                'Tranche 2': tranche2,
                'Zone 2': zone2,
                'Keep $': f"${keep_amount:,.0f}",
                'Keep %': f"{keep_pct:.0f}%"
            })
        
        if sell_actions:
            sheet2 = pd.DataFrame(sell_actions)
        else:
            sheet2 = pd.DataFrame([{'Ticker': 'No Sell Actions', 'Signal': 'No positions require reduction'}])
    else:
        sheet2 = pd.DataFrame([{'Ticker': 'No Sell Actions', 'Signal': 'No defensive signals'}])
    
    # === SHEET 3: TECHNICAL DATA (Full Dump) ===
    sheet3 = df.copy()
    
    # Write to Excel with formatting
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        sheet1.to_excel(writer, sheet_name='Buy Actions', index=False)
        sheet2.to_excel(writer, sheet_name='Sell Actions', index=False)
        sheet3.to_excel(writer, sheet_name='Technical Data', index=False)
        
        # Format each sheet
        for sheet_name in writer.sheets:
            worksheet = writer.sheets[sheet_name]
            
            # Freeze top row
            worksheet.freeze_panes = 'A2'
            
            # Auto-adjust column widths
            for column in worksheet.columns:
                max_length = 0
                column_letter = column[0].column_letter
                for cell in column:
                    try:
                        if len(str(cell.value)) > max_length:
                            max_length = len(str(cell.value))
                    except:
                        pass
                adjusted_width = min(max_length + 2, 50)
                worksheet.column_dimensions[column_letter].width = adjusted_width
    
    print(f"✅ Trading Actions Excel saved to Downloads: {output_file}")
    print(f"   📊 Sheet 1: {len(sheet1)} buy actions")
    print(f"   📊 Sheet 2: {len(sheet2)} sell actions")
    print(f"   📊 Sheet 3: {len(sheet3)} rows of technical data")

# Execute Excel generation
export_trading_actions_to_excel(df, starting_cash=104967)

✅ Trading Actions Excel saved to Downloads: C:\Users\karms\Downloads\trading_actions_20260105_1758.xlsx
   📊 Sheet 1: 5 buy actions
   📊 Sheet 2: 1 sell actions
   📊 Sheet 3: 15 rows of technical data


In [11]:
# ===================================================================================================
# CELL 10: CLEANUP OLD FILES IN DOWNLOADS
# ===================================================================================================
# Purpose: Automatically delete old report files, keep only 2 most recent of each type
# Inputs:  None (scans ~/Downloads directory)
# Outputs: Deletes old files, prints confirmation
# Files:   - trading_actions_*.xlsx
#          - buy_summary_*.pdf
#          - buy_detailed_*.pdf
#          - sell_summary_*.pdf
#          - sell_detailed_*.pdf
# Safety:  Skips files that are currently open (PermissionError handling)
# ===================================================================================================

from pathlib import Path

def cleanup_old_results(keep_latest=2):
    """Keep only the most recent batch result files in Downloads folder"""
    downloads_dir = Path.home() / 'Downloads'
    
    # Clean up trading actions Excel files
    excel_files = sorted(
        downloads_dir.glob('trading_actions_*.xlsx'),
        key=lambda p: p.stat().st_mtime,
        reverse=True
    )
    
    for old_file in excel_files[keep_latest:]:
        try:
            print(f"🗑️  Deleting old Excel: {old_file.name}")
            old_file.unlink()
        except PermissionError:
            print(f"⚠️  Skipped (file in use): {old_file.name}")
    
    # Clean up buy summary PDFs
    buy_pdfs = sorted(
        downloads_dir.glob('buy_summary_*.pdf'),
        key=lambda p: p.stat().st_mtime,
        reverse=True
    )
    
    for old_file in buy_pdfs[keep_latest:]:
        try:
            print(f"🗑️  Deleting old buy PDF: {old_file.name}")
            old_file.unlink()
        except PermissionError:
            print(f"⚠️  Skipped (file in use): {old_file.name}")
    
    # Clean up buy detailed PDFs
    buy_detailed_pdfs = sorted(
        downloads_dir.glob('buy_detailed_*.pdf'),
        key=lambda p: p.stat().st_mtime,
        reverse=True
    )
    
    for old_file in buy_detailed_pdfs[keep_latest:]:
        try:
            print(f"🗑️  Deleting old buy detailed PDF: {old_file.name}")
            old_file.unlink()
        except PermissionError:
            print(f"⚠️  Skipped (file in use): {old_file.name}")
    
    # Clean up sell summary PDFs
    sell_pdfs = sorted(
        downloads_dir.glob('sell_summary_*.pdf'),
        key=lambda p: p.stat().st_mtime,
        reverse=True
    )
    
    for old_file in sell_pdfs[keep_latest:]:
        try:
            print(f"🗑️  Deleting old sell PDF: {old_file.name}")
            old_file.unlink()
        except PermissionError:
            print(f"⚠️  Skipped (file in use): {old_file.name}")
    
    # Clean up sell detailed PDFs
    sell_detailed_pdfs = sorted(
        downloads_dir.glob('sell_detailed_*.pdf'),
        key=lambda p: p.stat().st_mtime,
        reverse=True
    )
    
    for old_file in sell_detailed_pdfs[keep_latest:]:
        try:
            print(f"🗑️  Deleting old sell detailed PDF: {old_file.name}")
            old_file.unlink()
        except PermissionError:
            print(f"⚠️  Skipped (file in use): {old_file.name}")
    
    print(f"✅ Cleanup complete: kept {keep_latest} most recent file(s) of each type")

# Auto-cleanup after each run:
cleanup_old_results(keep_latest=2)

✅ Cleanup complete: kept 2 most recent file(s) of each type
